In [16]:
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable



import torchvision
import torchvision.transforms as transforms

import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import csv

In [84]:
#https://github.com/maciejkula/spotlight/blob/master/spotlight/losses.py
def hinge_loss(positive_predictions, negative_predictions, mask=None):
    """
    Hinge pairwise loss function.
    Parameters
    ----------
    positive_predictions: tensor
        Tensor containing predictions for known positive items.
    negative_predictions: tensor
        Tensor containing predictions for sampled negative items.
    mask: tensor, optional
        A binary tensor used to zero the loss from some entries
        of the loss tensor.
    Returns
    -------
    loss, float
        The mean value of the loss function.
    """

    loss = torch.clamp(negative_predictions -
                       positive_predictions +
                       1.0, 0.0)

    if mask is not None:
        mask = mask.float()
        loss = loss * mask
        return loss.sum() / mask.sum()

    return loss.mean()

def adaptive_hinge_loss(positive_predictions, negative_predictions, mask=None):
    """
    Adaptive hinge pairwise loss function. Takes a set of predictions
    for implicitly negative items, and selects those that are highest,
    thus sampling those negatives that are closes to violating the
    ranking implicit in the pattern of user interactions.
    Approximates the idea of weighted approximate-rank pairwise loss
    introduced in [2]_
    Parameters
    ----------
    positive_predictions: tensor
        Tensor containing predictions for known positive items.
    negative_predictions: tensor
        Iterable of tensors containing predictions for sampled negative items.
        More tensors increase the likelihood of finding ranking-violating
        pairs, but risk overfitting.
    mask: tensor, optional
        A binary tensor used to zero the loss from some entries
        of the loss tensor.
    Returns
    -------
    loss, float
        The mean value of the loss function.
    References
    ----------
    .. [2] Weston, Jason, Samy Bengio, and Nicolas Usunier. "Wsabie:
       Scaling up to large vocabulary image annotation." IJCAI.
       Vol. 11. 2011.
    """

    highest_negative_predictions, _ = torch.max(negative_predictions, 0)

    return hinge_loss(positive_predictions, highest_negative_predictions.squeeze(), mask=mask)


In [25]:
books_ratings_df = pd.read_csv('data/BX-CSV-Dump/BX-Book-Ratings.csv',delimiter=';', encoding = "ISO-8859-1")
books_df = pd.read_csv('data/BX-CSV-Dump/BX-Books.csv',delimiter=';',encoding = "ISO-8859-1",quoting=csv.QUOTE_ALL,error_bad_lines=False )
users_df = pd.read_csv('data/BX-CSV-Dump/BX-Users.csv',delimiter=';',encoding = "ISO-8859-1",quoting=csv.QUOTE_ALL,error_bad_lines=False )

#include implicit recos only
books_ratings_implicit_df = books_ratings_df[books_ratings_df['Book-Rating'] == 0]
books_df = books_df[books_df['ISBN'].isin(books_ratings_implicit_df.ISBN.unique())]
users_df = users_df[users_df['User-ID'].isin(books_ratings_implicit_df['User-ID'].unique())]

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/Users/yehiaelshater/anaconda/envs/mypython3/lib/python3.7/site-packages/I

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
2,276727,0446520802,0
5,276733,2080674722,0
10,276746,0425115801,0
11,276746,0449006522,0


In [81]:
torch.manual_seed(1113)
#pos_predictions = torch.rand(5)
#neg_predictions = torch.rand(5)
pos_predictions = torch.tensor([3.2,3.2,4.9,4.9,-3.1,-3.1],dtype=torch.float32)
neg_predictions = torch.tensor([5.1,-1.7,1.3,2.0,2.5,2.2],dtype=torch.float32)
print(pos_predictions)
print(neg_predictions)
hinge_loss(pos_predictions,neg_predictions)

tensor([ 3.2000,  3.2000,  4.9000,  4.9000, -3.1000, -3.1000])
tensor([ 5.1000, -1.7000,  1.3000,  2.0000,  2.5000,  2.2000])


tensor(15.8000)

In [83]:
torch.clamp(neg_predictions -pos_predictions +1.0, 0.0).sum()

tensor(15.8000)